# Stacking and back propagation on Mnist with multiple sigmoids

### import data

In [67]:
from sklearn.datasets import fetch_mldata
import numpy as np
from sklearn.utils import shuffle
mnist = fetch_mldata('MNIST original', data_home=".")
mnist.data, mnist.target = shuffle(mnist.data, mnist.target)
X = mnist.data
y_float = mnist.target
y_int = y_float.astype(np.int64)
y = np.eye(10)[y_int]

### Cross Validation

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### Random forest

In [69]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
rf_train = clf.predict(X_train)
rf_test = clf.predict(X_test)

### add 1 to X

In [70]:
s_train = np.ones(X_train.shape[0])
X_train = np.hstack((X_train, rf_train, s_train.reshape(len(s_train),1)))
s_test = np.ones(X_test.shape[0])
X_test = np.hstack((X_test, rf_test, s_test.reshape(len(s_test),1)))

In [71]:
X_train.shape

(52500, 795)

### PCA

In [72]:
# from sklearn.decomposition import IncrementalPCA

# n_batches = 100
# inc_pca = IncrementalPCA(n_components = 154)
# for X_batch in np.array_split(X_train, n_batches):
#     inc_pca.partial_fit(X_batch)
    
# X_reduced_train = inc_pca.transform(X_train)
# X_reduced_train = np.hstack((X_reduced_train, s_train.reshape(len(s_train),1)))
# X_reduced_test = inc_pca.transform(X_test)
# X_reduced_test = np.hstack((X_reduced_test, s_test.reshape(len(s_test),1)))

### define function

In [73]:
def linear_output_z1(x, w1):
    z1 = np.dot(x, w1)
    return z1

def logistic_output_z2(x, w2):
    z2 = sigmoid(np.dot(x, w2))
    return z2

def sigmoid(a):
    return 1.0 / (1.0 + np.exp(-a))

### define gradient

In [74]:
def logistic_output(x, z1, z2, theta1, theta2, theta3):
    zf = np.matmul(z1, theta1) + np.matmul(z2, theta2) + np.array(np.dot(x, theta3))
    return zf

def gradient_c_theta1(z1, zf, y):
    gradient = np.matmul(np.transpose(z1), (sigmoid(zf)-y)) / N
    return gradient

def gradient_c_theta2(z2, zf, y):
    gradient = np.matmul(np.transpose(z2), (sigmoid(zf)-y)) / N
    return gradient

def gradient_c_theta3(x, zf, y):
    gradient = np.matmul(np.transpose(x), (sigmoid(zf)-y)) / N
    return gradient

def gradient_c_w1(x, theta1, zf, y):
    gradient = np.matmul(np.transpose(x), np.transpose(np.matmul(theta1, np.transpose(np.array((sigmoid(zf) - y)))))) / N
    return gradient

def gradient_c_w2(x, w2, theta2, zf, y):
    elemsig = sigmoid(np.matmul(x, w2)) # [5000, 1]
    elem1=np.transpose(np.array(sigmoid(zf) - y)) # [10,5000]
    elem2=np.matmul(theta2, elem1) # [1, 5000]
    elem3=np.matmul(np.transpose(elemsig), np.transpose(elem2)) # [1,1]
    elem4 = np.matmul(elemsig, (1 - elem3)) # [5000,1]
    gradient= np.matmul(np.transpose(x), elem4)/ N # [785,1]

    return gradient


In [75]:
# z1 = linear_output_z1(X_train, int_w1)
# z2 = logistic_output_z2(X_train, int_w2)
# zf = logistic_output(X_train, z1, z2, int_theta1, int_theta2, int_theta3)

# elemsig = sigmoid(np.matmul(X_train, int_w2)) # [52500, 2]
# elem1=np.transpose(np.array(sigmoid(zf) - y_train)) # [10,52500]
# elem2=np.matmul(int_theta2, elem1) # [2, 52500]
# elem3=np.matmul(np.transpose(elemsig), np.transpose(elem2)) # [2,2]
# elem4 = np.matmul(elemsig, (1 - elem3)) # [5000,1]
# # gradient= np.matmul(np.transpose(x), elem4)/ N # [785,1]

### define gradient descent 

In [76]:
def gradient_descent(x, y, int_w1, int_w2, int_theta1, int_theta2, int_theta3, lowtrain, uptrain, iterations):

    place_w1=int_w1
    place_w2=int_w2
    place_theta1=int_theta1
    place_theta2=int_theta2
    place_theta3=int_theta3

    for i in range(iterations):
        trainrate=np.random.uniform(lowtrain,uptrain,1)/np.cbrt(i+1) 
        z1 = linear_output_z1(x, place_w1)
        z2 = logistic_output_z2(x, place_w2)
        zf = logistic_output(x, z1, z2, place_theta1, place_theta2, place_theta3)
        p = sigmoid(zf)

        gradient_theta1 = gradient_c_theta1(z1, zf, y)
        gradient_theta2 = gradient_c_theta2(z2, zf, y)
        gradient_theta3 = gradient_c_theta3(x, zf, y)
        gradient_w1 = gradient_c_w1(x, place_theta1, zf, y)
        gradient_w2 = gradient_c_w2(x, place_w2, place_theta2, zf, y)
        
            
        place_w1 = place_w1 - (trainrate * gradient_w1) - (0.1/N)*place_w1 * trainrate
        place_w2 = place_w2 - (trainrate * gradient_w2) - (0.1/N)*place_w2 * trainrate
        place_theta1 = place_theta1 - trainrate*gradient_theta1*0.01 - (0.1/N)*place_theta1 * trainrate 
        place_theta2 = place_theta2 - trainrate*gradient_theta2 - (0.1/N)*place_theta2 * trainrate
        place_theta3 = place_theta3 - trainrate*gradient_theta3 - (0.1/N)*place_theta3 * trainrate

    
    return  p, place_theta1, place_theta2, place_theta3, place_w1, place_w2


### initialization

In [77]:
N_linear = 3
M_logistic = 3

int_w1, int_w2 = np.random.uniform(-1,1,(X_train.shape[1],N_linear)), np.random.uniform(-1,1,(X_train.shape[1], M_logistic))
int_theta1, int_theta2, int_theta3 = np.random.uniform(-1,1,(N_linear,10)), np.random.uniform(-1,1,(M_logistic,10)),np.random.uniform(-1,1,(X_train.shape[1],10))
result = gradient_descent(X_train, y_train, int_w1, int_w2, int_theta1, int_theta2, int_theta3, 0.00001, 0.0001, 1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


In [78]:
def predict_accuracy(xtest, ytest, theta1, theta2, theta3, w1, w2):
    z1 = linear_output_z1(xtest, w1)
    z2 = logistic_output_z2(xtest, w2)
    zf = logistic_output(xtest, z1, z2, theta1, theta2, theta3)
    p = sigmoid(zf)
    prediction = np.argmax(p, axis=1)
    answer = np.argmax(ytest, axis=1)
    success_rate = answer == prediction
    success_rate = success_rate*1
    prediction_rate = sum(success_rate) / xtest.shape[0]
    
    return prediction_rate

In [80]:
for i in range(500):
    X_train, y_train = shuffle(X_train, y_train)
    X_train_mini, y_train_mini = X_train[0:2000], y_train[0:2000]
    N = X_train_mini.shape[0]
    int_w1 = result[4]
    int_w2 = result[5]
    int_theta1 = result[1]
    int_theta2 = result[2]
    int_theta3 = result[3]

    result = gradient_descent(X_train_mini, y_train_mini, int_w1, int_w2, int_theta1, int_theta2, int_theta3, 0.00001, 0.0001, 10)
    output = predict_accuracy(X_test, y_test, result[1], result[2], result[3], result[4], result[5])
    print(output)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


0.800971428571
0.800628571429
0.8012
0.801142857143
0.800971428571
0.8012
0.801771428571
0.801028571429
0.801771428571
0.801428571429
0.801885714286
0.8012
0.802228571429
0.801942857143
0.802114285714
0.801942857143
0.802
0.802057142857
0.802571428571
0.803485714286
0.802742857143
0.801942857143
0.802571428571
0.8032
0.802857142857
0.802342857143
0.8028
0.803028571429
0.803714285714
0.803657142857
0.803542857143
0.803657142857
0.8032
0.802971428571
0.803371428571
0.803942857143
0.803885714286
0.8044
0.803828571429
0.803942857143
0.804228571429
0.804457142857
0.804971428571
0.804685714286
0.804742857143
0.804742857143
0.804514285714
0.803542857143
0.803828571429
0.804114285714
0.804114285714
0.804228571429
0.804171428571
0.804628571429
0.804057142857
0.803428571429
0.802971428571
0.803371428571
0.803428571429
0.803314285714
0.803028571429
0.803828571429
0.803942857143
0.8032
0.803771428571
0.8044
0.804457142857
0.804514285714
0.804742857143
0.8048
0.804742857143
0.805028571429
0.8048571

### save array

In [53]:
nparray = np.array(result)
np.save('result.npy', nparray)